In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow.compat.v2.feature_column as fc
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
import io
from tensorflow import keras
import seaborn as sns
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
import pickle as pk
from sklearn.decomposition import PCA as pca
!pip3 install pickle5
import pickle5 as pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#with open('/content/drive/MyDrive/dataframe_95.pkl', 'rb') as f:
#    clean_data = pk.load(f)
with open('/content/drive/MyDrive/95.pkl', 'rb') as f:
    clean_data = pickle.load(f)
print(clean_data.describe())



       icustay_seq           age  ...  days_to_death  number_of_icu_stays
count      33739.0  33739.000000  ...   14555.000000         33739.000000
mean           0.0      0.649946  ...     522.393748             1.073214
std            0.0      0.227717  ...     760.794850             0.300686
min            0.0      0.000000  ...       0.000000             1.000000
25%            0.0      0.500000  ...      18.000000             1.000000
50%            0.0      0.671053  ...     146.000000             1.000000
75%            0.0      0.828947  ...     743.000000             1.000000
max            0.0      1.000000  ...    4166.000000             6.000000

[8 rows x 1475 columns]


In [ ]:
clean_data.pop('died_in_hospital')
clean_data.pop('days_to_death')
clean_data.pop('length_of_stay_icu')
clean_data.pop('length_of_stay_hospital')

print(clean_data.columns[-2:])
clean_data.describe()


Index([1404, 'number_of_icu_stays'], dtype='object')


,icustay_seq,age,gender,weight,height,heartrate_mean,heartrate_min,heartrate_max,meanbp_mean,meanbp_min,meanbp_max,resprate_mean,resprate_min,resprate_max,tempc_mean,tempc_min,tempc_max,spo2_mean,spo2_min,spo2_max,glucose_mean,glucose_min,glucose_max,received_dialysis,received_ventilation,urineoutput,mingcs,gcsmotor,gcsverbal,gcseyes,aniongap_min,aniongap_max,albumin_min,albumin_max,bands_min,bands_max,bicarbonate_min,bicarbonate_max,bilirubin_min,bilirubin_max,...,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,number_of_icu_stays
count,33739.0,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,...,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000,33739.000000
mean,0.0,0.649946,0.561072,0.265993,0.150713,0.437387,0.505588,0.318804,0.445517,0.463398,0.295806,0.317772,0.338448,0.313568,0.608165,0.816517,0.421600,0.958897,0.914557,0.989452,0.120899,0.180773,0.162910,0.042740,0.434808,0.008083,0.896701,0.833464,0.638346,0.706250,0.270078,0.217330,0.456328,0.464743,0.106804,0.116482,0.401299,0.420580,0.020586,0.021083,...,0.459380,0.468737,0.371313,0.345642,0.349573,0.409571,0.357736,0.490945,0.442341,0.459824,0.477593,0.418463,0.489952,0.439938,0.389691,0.403905,0.417868,0.476415,0.437251,0.324176,0.467575,0.477647,0.470474,0.431312,0.431632,0.417936,0.496216,0.414522,0.365711,0.413388,0.486038,0.433024,0.443515,0.479993,0.478462,0.407352,0.425556,0.362917,0.366523,1.073214
std,0.0,0.227717,0.496264,0.072434,0.024277,0.125528,0.107246,0.099356,0.082203,0.109729,0.092473,0.107771,0.103445,0.106138,0.060697,0.030773,0.050241,0.037412,0.086208,0.029172,0.045223,0.058763,0.087458,0.202273,0.495739,0.005905,0.216548,0.323021,0.430116,0.368975,0.073317,0.086939,0.092495,0.089105,0.045288,0.047300,0.099200,0.091351,0.035195,0.034354,...,0.040445,0.049697,0.043045,0.029525,0.038844,0.040791,0.035237,0.050327,0.035954,0.045284,0.036636,0.042758,0.046228,0.047411,0.048100,0.038838,0.035166,0.042574,0.049702,0.032683,0.042091,0.044164,0.041525,0.044124,0.044071,0.043769,0.043118,0.039511,0.036585,0.043318,0.049161,0.033465,0.045671,0.044026,0.033961,0.038368,0.043651,0.043756,0.039926,0.300686
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.0,0.500000,0.000000,0.220736,0.150713,0.346774,0.434300,0.250000,0.386861,0.403226,0.242754,0.

In [ ]:
train_dataset = clean_data.sample(frac=0.8, random_state=0)
test_dataset = clean_data.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('number_of_icu_stays')
test_labels = test_features.pop('number_of_icu_stays')

print(train_dataset)
print(test_dataset)
#print(train_labels)
#print(test_labels)



       icustay_seq       age  gender  ...      1403      1404  number_of_icu_stays
12945          0.0  0.197368     0.0  ...  0.373869  0.309853                  1.0
10850          0.0  0.368421     1.0  ...  0.576260  0.304946                  1.0
7765           0.0  0.960526     1.0  ...  0.359185  0.363750                  2.0
30225          0.0  0.644737     0.0  ...  0.362117  0.357388                  1.0
13399          0.0  0.828947     1.0  ...  0.347737  0.386521                  1.0
...            ...       ...     ...  ...       ...       ...                  ...
4824           0.0  0.618421     1.0  ...  0.360196  0.403206                  1.0
14769          0.0  0.539474     1.0  ...  0.355608  0.369854                  1.0
8447           0.0  0.802632     0.0  ...  0.356990  0.392108                  1.0
11617          0.0  0.328947     1.0  ...  0.354211  0.556579                  1.0
15856          0.0  0.078947     0.0  ...  0.361771  0.367944                  1.0

[26

In [ ]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())




[0.         0.6492383  0.5650774  ... 0.4253737  0.36287412 0.36644712]


In [ ]:
METRICS=[tf.keras.metrics.MeanSquaredError(name="mean_squared_error", dtype=None)]    


In [ ]:
def build_and_compile_model(norm):
    model = keras.Sequential([
      norm,
      layers.Dense(16, activation='relu'),
      layers.Dense(16, activation='relu'),
      layers.Dense(1)
    ])

    
    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001),metrics=METRICS)
    return model



In [ ]:
dnn_model = build_and_compile_model(normalizer)
#dnn_model.summary()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='mean_squared_error', 
    verbose=1,
    patience=29,
    mode='max',
    restore_best_weights=True)

In [ ]:

History = dnn_model.fit(
        train_features, train_labels,
        validation_split=0.2, epochs=128,
        callbacks=[early_stopping],)


Epoch 1/128
675/675 [==============================] - 3s 3ms/step - loss: 0.3384 - mean_squared_error: 0.2990 - val_loss: 0.1553 - val_mean_squared_error: 0.1027
Epoch 2/128
675/675 [==============================] - 2s 2ms/step - loss: 0.1111 - mean_squared_error: 0.0950 - val_loss: 0.0862 - val_mean_squared_error: 0.0891
Epoch 3/128
675/675 [==============================] - 2s 2ms/step - loss: 0.0818 - mean_squared_error: 0.0928 - val_loss: 0.0791 - val_mean_squared_error: 0.0888
Epoch 4/128
675/675 [==============================] - 2s 2ms/step - loss: 0.0758 - mean_squared_error: 0.0894 - val_loss: 0.0802 - val_mean_squared_error: 0.0889
Epoch 5/128
675/675 [==============================] - 2s 2ms/step - loss: 0.0740 - mean_squared_error: 0.0864 - val_loss: 0.0791 - val_mean_squared_error: 0.0892
Epoch 6/128
675/675 [==============================] - 2s 2ms/step - loss: 0.0724 - mean_squared_error: 0.0835 - val_loss: 0.0796 - val_mean_squared_error: 0.0883
Epoch 7/128
675/675 [=